# 범주형 변수 처리 : kaggler LabelEncoder 활용

In [1]:
import numpy as np 
import pandas as pd
from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss 
from lightgbm import LGBMClassifier 

In [11]:
train = pd.read_csv('card_dataset/train.csv', index_col = 0)
test = pd.read_csv('card_dataset/test.csv', index_col = 0)

In [8]:
import kaggler
from kaggler.preprocessing import LabelEncoder

In [12]:
train.fillna('NAN', inplace =True)
test.fillna('NAN', inplace =True)

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26457 entries, 0 to 26456
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         26457 non-null  object 
 1   car            26457 non-null  object 
 2   reality        26457 non-null  object 
 3   child_num      26457 non-null  int64  
 4   income_total   26457 non-null  float64
 5   income_type    26457 non-null  object 
 6   edu_type       26457 non-null  object 
 7   family_type    26457 non-null  object 
 8   house_type     26457 non-null  object 
 9   DAYS_BIRTH     26457 non-null  int64  
 10  DAYS_EMPLOYED  26457 non-null  int64  
 11  FLAG_MOBIL     26457 non-null  int64  
 12  work_phone     26457 non-null  int64  
 13  phone          26457 non-null  int64  
 14  email          26457 non-null  int64  
 15  occyp_type     26457 non-null  object 
 16  family_size    26457 non-null  float64
 17  begin_month    26457 non-null  float64
 18  credit

In [14]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
index,,,,,,,,,,,,,,,,,,,
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NAN,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NAN,2.0,-47.0,2.0
26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0


In [9]:
lbe = LabelEncoder(min_obs = 10)

In [28]:
df =pd.concat([train.drop('credit', axis = 1), test], axis = 0 )
df.shape

(36457, 18)

In [29]:
target_col = 'credit'
cat_cols = [x for x in df.columns if df[x].dtype == 'object']
num_cols = [x for x in df.columns if x not in cat_cols + [target_col]]

In [30]:
cat_cols

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occyp_type']

In [31]:
num_cols

['child_num',
 'income_total',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'work_phone',
 'phone',
 'email',
 'family_size',
 'begin_month']

In [32]:
df[cat_cols] = lbe.fit_transform(df[cat_cols])

/Users/jihye/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/jihye/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [33]:
df[num_cols].describe()

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month
count,36457.000000,3.645700e+04,36457.000000,36457.000000,36457.0,36457.000000,36457.000000,36457.000000,36457.000000,36457.000000
mean,0.430315,1.866857e+05,-15975.173382,59262.935568,1.0,0.225526,0.294813,0.089722,2.198453,-26.164193
std,0.742367,1.017892e+05,4200.549944,137651.334859,0.0,0.417934,0.455965,0.285787,0.911686,16.501854
min,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000
25%,0.000000,1.215000e+05,-19438.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000
50%,0.000000,1.575000e+05,-15563.000000,-1552.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000
75%,1.000000,2.250000e+05,-12462.000000,-408.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000
max,19.000000,1.575000e+06,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000,0.000000


In [58]:
n_train = df.iloc[:train.shape[0]]
n_test = df.iloc[train.shape[0]:]

In [59]:
print(n_train.shape, n_test.shape)

(26457, 18) (10000, 18)


In [62]:
n_test.tail()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month
index,,,,,,,,,,,,,,,,,,
36452,2,3,2,0,202500.0,3,6,1,1,-18593,-5434,1,1,1,0,20,2.0,-19.0
36453,3,3,2,0,202500.0,3,3,7,1,-10886,-1315,1,1,0,0,6,2.0,-34.0
36454,2,2,2,0,292500.0,3,3,1,1,-21016,-14018,1,0,0,0,19,2.0,-55.0
36455,2,3,3,0,180000.0,5,3,1,1,-16541,-1085,1,0,1,0,2,2.0,-33.0
36456,2,2,2,0,270000.0,3,4,1,1,-9154,-187,1,0,0,1,6,2.0,-11.0


In [42]:
y = train['credit']
x = n_train.drop('FLAG_MOBIL', axis =1)
test_x = n_test.drop('FLAG_MOBIL', axis = 1 )

In [43]:
folds = StratifiedKFold(n_splits = 5, 
                       shuffle = True , random_state = 511)
smote = SMOTE(random_state = 0 )
X_train_over, y_train_over = smote.fit_sample(x, y)

In [44]:
loss = []
sub = np.zeros((test_x.shape[0], 3 ))


for n_folds, (train_index, val_index) in enumerate(folds.split(X_train_over, y_train_over)):
    
    print(f'======================================={n_folds + 1 }==========================================')
    
    X_train, X_val = X_train_over.iloc[train_index], X_train_over.iloc[val_index]
    y_train, y_val = y_train_over.iloc[train_index], y_train_over.iloc[val_index]
    
    
    lgb = LGBMClassifier(n_estimators = 10000,
                        n_jobs = -1, 
                        num_leaves = 2^20,
                        max_depth = 32,
                        random_state = 512,
                        learning_rate = 0.03)
    
    lgb.fit(X_train, y_train,
           eval_set = [(X_train, y_train), (X_val, y_val)],
           early_stopping_rounds = 100,
           verbose  = 1000)
    
    y_val_onehot = pd.get_dummies(y_val)
    prediction = lgb.predict_proba(X_val)
    loss.append(log_loss(y_val_onehot, prediction))
    
    sub+= lgb.predict_proba(test_x)
    
    
    print(f'=============================================================================================\n\n')

=======================================1==========================================
Training until validation scores don't improve for 100 rounds
[1000]	training's multi_logloss: 0.557325	valid_1's multi_logloss: 0.632852
[2000]	training's multi_logloss: 0.434148	valid_1's multi_logloss: 0.563826
[3000]	training's multi_logloss: 0.355837	valid_1's multi_logloss: 0.532412
[4000]	training's multi_logloss: 0.300004	valid_1's multi_logloss: 0.513936
[5000]	training's multi_logloss: 0.257296	valid_1's multi_logloss: 0.505082
[6000]	training's multi_logloss: 0.222677	valid_1's multi_logloss: 0.50009
Early stopping, best iteration is:
[6323]	training's multi_logloss: 0.212336	valid_1's multi_logloss: 0.498678


=======================================2==========================================
Training until validation scores don't improve for 100 rounds
[1000]	training's multi_logloss: 0.564151	valid_1's multi_logloss: 0.642031
[2000]	training's multi_logloss: 0.436015	valid_1's multi_logloss:

In [45]:
np.mean(loss)

0.4974798905865154

In [51]:
submission = pd.read_csv('card_dataset/sample_submission.csv') 

In [52]:
submission

,index,0,1,2
0,26457,0,0,0
1,26458,0,0,0
2,26459,0,0,0
3,26460,0,0,0
4,26461,0,0,0
...,...,...,...,...
9995,36452,0,0,0
9996,36453,0,0,0
9997,36454,0,0,0
9998,36455,0,0,0


In [57]:
sub/5

array([[0.14674849, 0.59954852, 0.25370299],
       [0.20719685, 0.7854785 , 0.00732465],
       [0.28851575, 0.70714762, 0.00433664],
       ...,
       [0.39174453, 0.55151163, 0.05674384],
       [0.57637921, 0.42260256, 0.00101824],
       [0.02593477, 0.97237748, 0.00168775]])

In [54]:
submission.iloc[:,1:] = sub/5

In [55]:
submission.to_csv('submission/kagglerLBE.csv', index =False)